In [1]:
import pickle
from collections import Counter

import pandas as pd

In [2]:
# with open("../data/model.pkl", "rb") as f:
#     model = pickle.load(f)

with open("../data/retrain_model.pkl", "rb") as f:
    model = pickle.load(f)

In [3]:
df = pd.read_pickle("../data/trainset.pkl")
anime_list = df[["Name", "Japanese_name", "anime_id", "Score"]].drop_duplicates()

### Qualitative evaluation

- Pick up random user


In [4]:
def search_anime(df, search_word):
    result = df[df["Name"].str.contains(search_word, case=False)]
    print(result[["anime_id", "Name"]].values)


def show_random_user(df, iid):
    return df[(df["anime_id"] == iid) & (df["rating"] == 10)].head(10)


def show_user_info(df, uid):
    return df[df["user_id"] == uid].sort_values("rating", ascending=False).head(10)


def predict(uid, df, anime_list):
    # predict recommend item
    result_df = model.rank(user=uid, items=anime_list["anime_id"])
    result_df = pd.DataFrame(result_df, columns=["anime_id"])
    result_df = pd.merge(result_df, anime_list, on=["anime_id"], how="inner")

    # remove rated item
    rated_df = df[df["user_id"] == uid]["anime_id"]
    result_df = result_df[~result_df["anime_id"].isin(rated_df)]
    result_df = result_df.reset_index(drop=True)[0:10]

    return result_df

#### ジャンプ系


In [5]:
search_anime(anime_list, "kimetsu")

[[38000 'Kimetsu no Yaiba']]


In [6]:
show_random_user(df, 38000)

,user_id,anime_id,rating,Name,English_name,Japanese_name,Aired,Score,Ranked
458313,20,38000,10,Kimetsu no Yaiba,Demon Slayer:Kimetsu no Yaiba,鬼滅の刃,"Apr 6, 2019 to Sep 28, 2019",8.62,62.0
458314,25,38000,10,Kimetsu no Yaiba,Demon Slayer:Kimetsu no Yaiba,鬼滅の刃,"Apr 6, 2019 to Sep 28, 2019",8.62,62.0
458317,47,38000,10,Kimetsu no Yaiba,Demon Slayer:Kimetsu no Yaiba,鬼滅の刃,"Apr 6, 2019 to Sep 28, 2019",8.62,62.0
458324,82,38000,10,Kimetsu no Yaiba,Demon Slayer:Kimetsu no Yaiba,鬼滅の刃,"Apr 6, 2019 to Sep 28, 2019",8.62,62.0
458325,83,38000,10,Kimetsu no Yaiba,Demon Slayer:Kimetsu no Yaiba,鬼滅の刃,"Apr 6, 2019 to Sep 28, 2019",8.62,62.0
458327,110,38000,10,Kimetsu no Yaiba,Demon Slayer:Kimetsu no Yaiba,鬼滅の刃,"Apr 6, 2019 to Sep 28, 2019",8.62,62.0
458328,111,38000,10,Kimetsu no Yaiba,Demon Slayer:Kimetsu no Yaiba,鬼滅の刃,"Apr 6, 2019 to Sep 28, 2019",8.62,62.0
458331,120,38000,10,Kimetsu no Yaiba,Demon Slayer:Kimetsu no Yaiba,鬼滅の刃,"Apr 6, 2019 to Sep 28, 2019",8.62,62.0
458335,136,38000,10,Kimetsu no Yaiba,Demon Slayer:Kimetsu no Yaiba,鬼滅の刃,"Apr 6, 2019 to Sep 28, 2019",8.62,62.0
458337,139,38000,10,Kimetsu no Yaiba,Demon Slayer:Kimetsu no Yaiba,鬼滅の刃,"Apr 6, 2019 to Sep 28, 2019",8.62,62.0


In [7]:
show_user_info(df, 20)

,user_id,anime_id,rating,Name,English_name,Japanese_name,Aired,Score,Ranked
31155246,20,39533,10,Given,Unknown,ギヴン,"Jul 12, 2019 to Sep 20, 2019",8.32,214.0
2465104,20,35849,10,Darling in the FranXX,DARLING in the FRANXX,ダーリン・イン・ザ・フランキス,"Jan 13, 2018 to Jul 7, 2018",7.33,2210.0
7884425,20,35790,10,Tate no Yuusha no Nariagari,The Rising of the Shield Hero,盾の勇者の成り上がり,"Jan 9, 2019 to Jun 26, 2019",8.00,537.0
4571650,20,30276,10,One Punch Man,One Punch Man,ワンパンマン,"Oct 5, 2015 to Dec 21, 2015",8.57,81.0
458313,20,38000,10,Kimetsu no Yaiba,Demon Slayer:Kimetsu no Yaiba,鬼滅の刃,"Apr 6, 2019 to Sep 28, 2019",8.62,62.0
3877566,20,37430,10,Tensei shitara Slime Datta Ken,That Time I Got Reincarnated as a Slime,転生したらスライムだった件,"Oct 2, 2018 to Mar 19, 2019",8.07,458.0
2901207,20,28891,10,Haikyuu!! Second Season,Haikyu!! 2nd Season,ハイキュー!! セカンドシーズン,"Oct 4, 2015 to Mar 27, 2016",8.73,38.0
19683615,20,552,10,Digimon Adventure,Digimon:Digital Monsters,デジモンアドベンチャー,"Mar 7, 1999 to Mar 26, 2000",7.79,839.0
24611509,20,32615,10,Youjo Senki,The Saga of Tanya the Evil,幼女戦記,"Jan 6, 2017 to Mar 31, 2017",8.00,547.0
1475988,20,32935,10,Haikyuu!!: Karasuno Koukou vs. Shiratorizawa G...,Haikyu!! 3rd Season,ハイキュー!! 烏野高校 VS 白鳥沢学園高校,"Oct 8, 2016 to Dec 10, 2016",8.87,20.0


In [8]:
predict(20, df, anime_list)

,anime_id,Name,Japanese_name,Score
0,820,Ginga Eiyuu Densetsu,銀河英雄伝説,9.07
1,3297,Aria the Origination,ARIA The ORIGINATION,8.53
2,10937,Mobile Suit Gundam: The Origin,機動戦士ガンダムTHE ORIGIN,8.36
3,28977,Gintama°,銀魂°,9.10
4,3371,Ginga Eiyuu Densetsu Gaiden,銀河英雄伝説外伝／千億の星、千億の光,8.15
5,44,Rurouni Kenshin: Meiji Kenkaku Romantan - Tsui...,るろうに剣心―明治剣客浪漫譚―追憶編,8.73
6,35180,3-gatsu no Lion 2nd Season,3月のライオン 第2シリーズ,9.00
7,918,Gintama,銀魂,8.96
8,2921,Ashita no Joe 2,あしたのジョー２,8.67
9,21939,Mushishi Zoku Shou,蟲師 続章,8.72


#### 女性向け


In [9]:
search_anime(anime_list, "inu")

[[34542 'Inuyashiki']
 [249 'InuYasha']
 [17831 'Inu to Hasami wa Tsukaiyou']
 [22123 'Inugami-san to Nekoyama-san']
 [11013 'Inu x Boku SS']
 [39523
  'Choujin Koukousei-tachi wa Isekai demo Yoyuu de Ikinuku you desu!']
 [940 'Inukami!']
 [6811 'InuYasha: Kanketsu-hen']
 [8449 'Togainu no Chi']
 [13141 'Shiba Inuko-san']
 [31387 'Aru Hi Inu no Kuni kara Tegami ga Kite']
 [37890 'Oshi ga Budoukan Ittekuretara Shinu']
 [2553 'Flanders no Inu']
 [8920 'Inuki Kanako Zekkyou Collection: Gakkou ga Kowai!']
 [2749 'Heisei Inu Monogatari Bow']
 [2624 'Flanders no Inu, Boku no Patrasche']
 [1839 'Jagainu-kun']]


In [10]:
show_random_user(df, 11013)

,user_id,anime_id,rating,Name,English_name,Japanese_name,Aired,Score,Ranked
28533638,61,11013,10,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,"Jan 13, 2012 to Mar 30, 2012",7.46,1693.0
28533643,90,11013,10,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,"Jan 13, 2012 to Mar 30, 2012",7.46,1693.0
28533645,110,11013,10,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,"Jan 13, 2012 to Mar 30, 2012",7.46,1693.0
28533657,267,11013,10,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,"Jan 13, 2012 to Mar 30, 2012",7.46,1693.0
28533658,281,11013,10,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,"Jan 13, 2012 to Mar 30, 2012",7.46,1693.0
28533661,286,11013,10,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,"Jan 13, 2012 to Mar 30, 2012",7.46,1693.0
28533663,310,11013,10,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,"Jan 13, 2012 to Mar 30, 2012",7.46,1693.0
28533664,320,11013,10,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,"Jan 13, 2012 to Mar 30, 2012",7.46,1693.0
28533670,351,11013,10,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,"Jan 13, 2012 to Mar 30, 2012",7.46,1693.0
28533682,457,11013,10,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,"Jan 13, 2012 to Mar 30, 2012",7.46,1693.0


In [11]:
show_user_info(df, 457)

,user_id,anime_id,rating,Name,English_name,Japanese_name,Aired,Score,Ranked
263210,457,2034,10,Lovely★Complex,Lovely Complex,ラブ★コン,"Apr 7, 2007 to Sep 29, 2007",8.06,472.0
28533682,457,11013,10,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,"Jan 13, 2012 to Mar 30, 2012",7.46,1693.0
30162955,457,18661,10,Kamisama Hajimemashita OVA,Unknown,神様はじめました,"Aug 20, 2013",7.98,560.0
17777817,457,1562,10,Yamato Nadeshiko Shichihenge♥,The Seven Metamorphoses of Yamato Nadeshiko,ヤマトナデシコ七変化♥,"Oct 4, 2006 to Mar 28, 2007",7.76,902.0
6058331,457,23755,10,Nanatsu no Taizai,The Seven Deadly Sins,七つの大罪,"Oct 5, 2014 to Mar 29, 2015",7.89,690.0
17643152,457,120,10,Fruits Basket,Fruits Basket,フルーツバスケット,"Jul 5, 2001 to Dec 27, 2001",7.69,1058.0
6314796,457,22319,10,Tokyo Ghoul,Tokyo Ghoul,東京喰種-トーキョーグール-,"Jul 4, 2014 to Sep 19, 2014",7.81,807.0
29993289,457,25781,10,Shingeki no Kyojin: Kuinaki Sentaku,Unknown,進撃の巨人 悔いなき選択,"Dec 9, 2014 to Apr 9, 2015",8.38,170.0
33961906,457,14645,10,Hiiro no Kakera Dai Ni Shou,Hiiro no Kakera:The Tamayori Princess Saga Sea...,緋色の欠片 第二章,"Sep 30, 2012 to Dec 23, 2012",7.23,2651.0
8469589,457,14227,10,Tonari no Kaibutsu-kun,My Little Monster,となりの怪物くん,"Oct 2, 2012 to Dec 25, 2012",7.55,1417.0


In [12]:
predict(457, df, anime_list)

,anime_id,Name,Japanese_name,Score
0,820,Ginga Eiyuu Densetsu,銀河英雄伝説,9.07
1,35180,3-gatsu no Lion 2nd Season,3月のライオン 第2シリーズ,9.00
2,3297,Aria the Origination,ARIA The ORIGINATION,8.53
3,28735,Shouwa Genroku Rakugo Shinjuu,昭和元禄落語心中,8.60
4,33095,Shouwa Genroku Rakugo Shinjuu: Sukeroku Futata...,昭和元禄落語心中～助六再び篇～,8.79
5,40417,Fruits Basket 2nd Season,フルーツバスケット 2nd season,8.54
6,338,Versailles no Bara,ベルサイユのばら,8.33
7,1,Cowboy Bebop,カウボーイビバップ,8.78
8,36649,Banana Fish,BANANA FISH,8.45
9,5420,Kemono no Souja Erin,獣の奏者エリン,8.35


- 人気作品に引っ張られる問題
- ベルばら、BANANA FISH、獣の奏者エリンはいい感じな気がする
